<h1 align="center"> pyLDPC Tutorial: Images </h1>


## update:  03/28/16 - v.0.8.0

<b><font color="red"> Since version 0.7: Coding and decoding functions take tG (Transposed G) instead of G, the coding matrix. Functions that construct it (CodingMatrix and CodingMatrix_systematic) return tG instead of G as well. </font></b> 

<b><font color="red"> Since version 0.8: Image coding and decoding functions are suitable for any LDPC code of any size, with out  restrictions  </font></b> 

<br>
<b><font color="blue"> New functions/sections are indicated in blue </font></b>



This notebook introduces a user's guide of pyLDPC's sub-module Images: ldpc_images.
If you would like to know what each function does and go into the construction details, go to <a href="http://nbviewer.jupyter.org/github/janatiH/pyldpc/blob/master/pyLDPC-Images-Construction.ipynb?flush_cache=true/"> LDPC-Images Construction Details</a>


<font color="blue"><h2> Outline:</h2></font> 
<font color="blue"><h2> --------------------------------------------------</h2></font> 

**I. Step 1: Construct Code **

**II. Step 2: Read and binarize the image **

**III. Step 3: Code the image and vizualise the noise **

**IV. Step 4: Decode the transmitted image **

<font color="blue"><h2> --------------------------------------------------</h2></font> 

We'll consider 2 types of images in this tutorial:

- Grayscale images: Images whean read by cv2.imread(), are seen as 2D arrays. Each pixel is a uint8 number (0-255). 
- RGB Images: Images that are seen as 3D arrays. Each pixel is an RGB array of 3 uint8 numbers ( [R,G,B] ). 


In [3]:
import numpy as np
from IPython.display import display, HTML
import pyldpc
from pyldpc import ldpc_images
import cv2 
from time import time
from scipy.sparse import csr_matrix


If I had to sum up Images Coding and decoding in a few words, I'd say:

1. Make coding and decoding matrices G and H.
2. Read the image [2D-array if grayscale, 3D-array if RGB]. 
3. Binarize the image [Using functions we'll see later].
4. Apply Coding function to binary image.
5. Apply Decoding function to Coded image. 

We'll go step by step 1 through 5 for both grayscale and RGB Images.

# Step 1: Construct a code


To construct a code you need to choose a the triplet (n,d_v,d_c). 
Keep in mind that: 

- the rate of the code k/n is approximately equal to 1 - d_v/d_c.****
- if high values of k,n are used, transform H and G to csr format using scipy.sparse.csr_matrix(): It's a compressed format with which calculations are faster. 


* *Details about how to choose matrices parameters <a href="http://nbviewer.jupyter.org/github/janatiH/pyldpc/blob/master/pyLDPC-Tutorial-Matrices.ipynb?flush_cache=true/">here </a>, but I recommand going through this tutorial and using the matrices defined hereby (or any other matrices) for now, and then optimize your coding & decoding by changing the matrices.*

In [5]:
n = 800
d_c = 4
d_v = 3 #In this case k is approximately equal to n/4 (n x (1-3/4))

H = pyldpc.RegularH(n,d_v,d_c)
H,tG = pyldpc.CodingMatrix_systematic(H)

tGs = csr_matrix(tG)
Hs = csr_matrix(H)

k = tG.shape[1]
k

202

# Step 2: Read and binarize the image 

Picture the image as a huge array of uint8 numbers (meaning: **u** for unsigned (positive) **int** for integer and 8 for 8 binary bits, which means that the number is lower than 2^7). 

If the image is grayscale, each pixel equals one uint8 number. The image is therefore read as a 2D-array. 
If the image is colorful, each pixel equals a tuple of 3 uint8 numbers (one for each color, Red Green Blue). The image is therefore a 3D-array, the third dimension being equal to 3. 

- If the image is grayscale, we write each pixel as a 8-sized binary array and pass it to coding and decoding functions. 
- If the image is RGB, we write each pixel as a 24-sized binary array and pass it to coding and decoding functions. 


Let's code and decode the grayscale image: <img src="Images/eye.png"> and its rgb form: <img src="Images/oeil.png">


Load the image using cv2.imread:

<b><font color="red"> imread's second argument is important: 0 for grayscale images, 1 for RGB images. 

In [7]:
eye_gray = cv2.imread("Images/oeil.png",0)
eye_gray.shape

(128, 128)

In [8]:
eye_rgb = cv2.imread("Images/oeil.png",1)
eye_rgb.shape

(128, 128, 3)

Transform image format from np.uint8 to binary: 

In [13]:
eye_gray_bin = ldpc_images.Gray2Bin(eye_gray)
eye_gray_bin.shape

(128, 128, 8)

In [14]:
eye_rgb_bin = ldpc_images.RGB2Bin(eye_rgb)
eye_rgb_bin.shape

(128, 128, 24)

# Step 3: Code the binary image
Code and send binary image eye_bin through a channel with:

- SNR = 6 db

The Coding function *ImageCoding* returns the coded and the noisy image:

- The coded image is passed later on to the decoding function.
- The noisy image is transformed to Gray or RGB to be vizualised. 



In [16]:
CODING = 1
snr = 6

### Grayscale: 

In [19]:
if CODING:
    eye_gray_coded,eye_gray_noisy_bin = ldpc_images.ImageCoding(tGs,eye_gray_bin,snr)
    
    eye_gray_noisy = ldpc_images.Bin2Gray(eye_gray_noisy_bin)
    
    ## Save the image locally: 
    cv2.imwrite("Images/eye/grayscale/snr=6/eye_noisy.jpg",eye_gray_noisy)


### RGB:

In [22]:
if CODING:
    eye_rgb_coded,eye_rgb_noisy_bin = ldpc_images.ImageCoding(tGs,eye_rgb_bin,snr)

    eye_rgb_noisy = ldpc_images.Bin2RGB(eye_rgb_noisy_bin)

    ## Save the image locally: 
    cv2.imwrite("Images/eye/rgb/snr=6/eye_noisy.jpg",eye_rgb_noisy)

### Let's have a look at  the noisy pictures:

In [20]:
display(HTML('''<img src="Images/eye/grayscale/snr=6/eye_noisy.jpg">'''))
display(HTML('''<img src="Images/eye/rgb/snr=6/eye_noisy.jpg">'''))

# Step 3: Decoding 
And now, let's decode:

<font color=#A44057> Warning ! Decoding takes time depending on the image's and the matrices' sizes.! </font>



> Iteration number: 1

In [23]:
DECODING = 1
max_iter = 1

### Grayscale

In [24]:
if DECODING:
    
    t = time()
    eye_gray_decoded_bin = ldpc_images.ImageDecoding(tGs,H,eye_gray_coded,snr,max_iter)
    t = time() - t
    
    eye_gray_decoded = ldpc_images.Bin2Gray(eye_gray_decoded_bin)
    
    cv2.imwrite("Images/eye/grayscale/snr=6/iterations/eye_iter"+str(max_iter)+".jpg",eye_gray_decoded)
    
    print("Decoding time for {} iterations and snr = {}: ".format(max_iter,snr),t) 
    
    display(HTML('''<img src="Images/eye/grayscale/snr=6/iterations/eye_iter'''+str(max_iter)+'''.jpg">'''))
    
print("Bit Error Rate in %:", 100*ldpc_images.BER(eye_gray_bin,eye_gray_decoded_bin),"%")

Decoding time for 1 iterations and snr = 6:  35.2626678943634


Bit Error Rate in %: 0.0602722167969 %


The decoding is not error-free, let's increment our decoding loop to 10 iterations: 

In [25]:
max_iter= 10

if DECODING:
    
    t = time()
    eye_gray_decoded_bin = ldpc_images.ImageDecoding(tGs,H,eye_gray_coded,snr,max_iter)
    t = time() - t
    
    eye_gray_decoded = ldpc_images.Bin2Gray(eye_gray_decoded_bin)
    
    cv2.imwrite("Images/eye/grayscale/snr=6/iterations/eye_iter"+str(max_iter)+".jpg",eye_gray_decoded)
    
    print("Decoding time for {} iterations and snr = {}: ".format(max_iter,snr),t) 
    
    display(HTML('''<img src="Images/eye/grayscale/snr=6/iterations/eye_iter'''+str(max_iter)+'''.jpg">'''))
    
print("Bit Error Rate in %:", 100*ldpc_images.BER(eye_gray_bin,eye_gray_decoded_bin),"%")

Decoding time for 10 iterations and snr = 6:  46.056742906570435


Bit Error Rate in %: 0.0 %


Decoding is errror-free !

### RGB 

In [26]:
DECODING = 1
max_iter = 1

In [27]:
if DECODING:
    
    t = time()
    eye_rgb_decoded_bin = ldpc_images.ImageDecoding(tGs,H,eye_rgb_coded,snr,max_iter)
    t = time() - t
    
    eye_rgb_decoded = ldpc_images.Bin2RGB(eye_rgb_decoded_bin)
    
    cv2.imwrite("Images/eye/grayscale/snr=6/iterations/eye_iter"+str(max_iter)+".jpg",eye_rgb_decoded)
    
    print("Decoding time for {} iterations and snr = {}: ".format(max_iter,snr),t) 
    
    display(HTML('''<img src="Images/eye/grayscale/snr=6/iterations/eye_iter'''+str(max_iter)+'''.jpg">'''))
    
print("Bit Error Rate in %:", 100*ldpc_images.BER(eye_rgb_bin,eye_rgb_decoded_bin),"%")

Decoding time for 1 iterations and snr = 6:  110.33554196357727


Bit Error Rate in %: 0.0554402669271 %


The decoding is not error-free, let's increment our decoding loop to 10 iterations: 

In [28]:
max_iter = 10 

if DECODING:
    
    t = time()
    eye_rgb_decoded_bin = ldpc_images.ImageDecoding(tGs,H,eye_rgb_coded,snr,max_iter)
    t = time() - t
    
    eye_rgb_decoded = ldpc_images.Bin2RGB(eye_rgb_decoded_bin)
    
    cv2.imwrite("Images/eye/grayscale/snr=6/iterations/eye_iter"+str(max_iter)+".jpg",eye_rgb_decoded)
    
    print("Decoding time for {} iterations and snr = {}: ".format(max_iter,snr),t) 
    
    display(HTML('''<img src="Images/eye/grayscale/snr=6/iterations/eye_iter'''+str(max_iter)+'''.jpg">'''))
    
print("Bit Error Rate in %:", 100*ldpc_images.BER(eye_rgb_bin,eye_rgb_decoded_bin),"%")

Decoding time for 10 iterations and snr = 6:  128.7423050403595


Bit Error Rate in %: 0.0 %


Error-free decoding ! 

